**This notebook is an exercise in the [Intermediate Machine Learning](https://www.kaggle.com/learn/intermediate-machine-learning) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/pipelines).**

---


In this exercise, you will use **pipelines** to improve the efficiency of your machine learning code.

# Setup

The questions below will give you feedback on your work. Run the following cell to set up the feedback system.

In [2]:
# Set up code checking
import os
if not os.path.exists("../input/train.csv"):
    os.symlink("../input/home-data-for-ml-course/train.csv", "../input/train.csv")  
    os.symlink("../input/home-data-for-ml-course/test.csv", "../input/test.csv") 
from learntools.core import binder
binder.bind(globals())
from learntools.ml_intermediate.ex4 import *
print("Setup Complete")

Setup Complete


You will work with data from the [Housing Prices Competition for Kaggle Learn Users](https://www.kaggle.com/c/home-data-for-ml-course). 

![Ames Housing dataset image](https://storage.googleapis.com/kaggle-media/learn/images/lTJVG4e.png)

Run the next code cell without changes to load the training and validation sets in `X_train`, `X_valid`, `y_train`, and `y_valid`.  The test set is loaded in `X_test`.

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X_full = pd.read_csv('../input/train.csv', index_col='Id')
X_test_full = pd.read_csv('../input/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

In [4]:
# target = {'ex', 'gd', 'ta', 'fa'}
# for col in X_train.select_dtypes(include='object').columns:
#     uniques = {str(v).strip().lower() for v in X_train[col].dropna().unique()}
#     if uniques & target:
#         extras = uniques - target
#         print(f"{col:20} | found: {sorted(uniques & target)} | extras: {sorted(extras) if extras else 'none'}")
# print("-------------------------------------------------------------------------------------------------------")
# target = {'lvl', 'bnk', 'hls', 'low'}
# # and keep the rest of your code the same
# for col in X_train.select_dtypes(include='object').columns:
#     uniques = {str(v).strip().lower() for v in X_train[col].dropna().unique()}
#     if uniques & target:
#         extras = uniques - target
#         print(f"{col:20} | found: {sorted(uniques & target)} | extras: {sorted(extras) if extras else 'none'}")
# print("-------------------------------------------------------------------------------------------------------")
# target = {'gtl', 'mod', 'sev'}
# # and keep the rest of your code the same
# for col in X_train.select_dtypes(include='object').columns:
#     uniques = {str(v).strip().lower() for v in X_train[col].dropna().unique()}
#     if uniques & target:
#         extras = uniques - target
#         print(f"{col:20} | found: {sorted(uniques & target)} | extras: {sorted(extras) if extras else 'none'}")
# print("-------------------------------------------------------------------------------------------------------")
# target = {'n', 'y'}
# # and keep the rest of your code the same
# for col in X_train.select_dtypes(include='object').columns:
#     uniques = {str(v).strip().lower() for v in X_train[col].dropna().unique()}
#     if uniques & target:
#         extras = uniques - target
#         print(f"{col:20} | found: {sorted(uniques & target)} | extras: {sorted(extras) if extras else 'none'}")

#print(X_train['ExterQual'].value_counts())
# print(X_train['PavedDrive'].value_counts())

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning, module="pandas.io.formats.format")
pd.set_option('display.max_columns', None)
X_train.head()
# print(X_train.iloc[:,70:80].head(40))

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
619,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,1Fam,1Story,Hip,CompShg,BrkFace,Ex,TA,PConc,Ex,TA,Av,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Gd,Attchd,Unf,TA,TA,Y,NaN,NaN,NaN,New,Partial,20,90.0,11694,9,5,2007,2007,452.0,48,0,1774,1822,1828,0,0,1828,0,0,2,0,3,1,9,1,2007.0,3,774,0,108,0,0,260,0,0,7,2007
871,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,PosN,Norm,1Fam,1Story,Hip,CompShg,NaN,TA,TA,CBlock,TA,TA,No,Unf,Unf,GasA,Gd,N,SBrkr,TA,Typ,NaN,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Normal,20,60.0,6600,5,5,1962,1962,0.0,0,0,894,894,894,0,0,894,0,0,1,0,2,1,5,0,1962.0,1,308,0,0,0,0,0,0,0,8,2009
93,RL,Pave,Grvl,IR1,HLS,AllPub,Inside,Gtl,Norm,Norm,1Fam,1Story,Gable,CompShg,NaN,TA,Gd,BrkTil,Gd,TA,No,ALQ,Unf,GasA,Ex,Y,SBrkr,TA,Typ,NaN,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Normal,30,80.0,13360,5,7,1921,2006,0.0,713,0,163,876,964,0,0,964,1,0,1,0,2,1,5,0,1921.0,2,432,0,0,44,0,0,0,0,8,2009
818,RL,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Norm,Norm,1Fam,1Story,Hip,CompShg,BrkFace,Gd,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Gd,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal,20,NaN,13265,8,5,2002,2002,148.0,1218,0,350,1568,1689,0,0,1689,1,0,2,0,3,1,7,2,2002.0,3,857,150,59,0,0,0,0,0,7,2008
303,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,1Fam,1Story,Gable,CompShg,BrkFace,Gd,TA,PConc,Gd,TA,No,Unf,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal,20,118.0,13704,7,5,2001,2002,150.0,0,0,1541,1541,1541,0,0,1541,0,0,2,0,3,1,6,1,2001.0,3,843,468,81,0,0,0,0,0,1,2006


The next code cell uses code from the tutorial to preprocess the data and train a model.  Run this code without changes.

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)

print('MAE:', mean_absolute_error(y_valid, preds))

MAE: 17614.81993150685


The code yields a value around 17862 for the mean absolute error (MAE).  In the next step, you will amend the code to do better.

# Step 1: Improve the performance

### Part A

Now, it's your turn!  In the code cell below, define your own preprocessing steps and random forest model.  Fill in values for the following variables:
- `numerical_transformer`
- `categorical_transformer`
- `model`

To pass this part of the exercise, you need only define valid preprocessing steps and a random forest model.

In [16]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='mode') # Your code here

# Preprocessing for categorical data
#========================================================================================================
# Old Approach 
#--------------------------------------------------------------------------------------------------------
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
# #========================================================================================================
# # New Approach 
# #--------------------------------------------------------------------------------------------------------
# # Bundle preprocessing for numerical and categorical data
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, FunctionTransformer
# import pandas as pd
# import numpy as np

# # ────────────────────────────────────────────────────────────────
# #  Helper mapping dictionaries (each rule is independent — comment to disable)
# # ────────────────────────────────────────────────────────────────

# ordinal_mappings = {
#     # 1. Ex Gd TA Fa Po → 5 4 3 2 1
#     'ex_gd_ta_fa_po': {
#         'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1,
#         np.nan: 0, 'NA': 0, None: 0
#     },

#     # 2. Basement exposure: Gd Av Mn No NA → 4 3 2 1 0
#     'bsmt_exposure': {
#         'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, 'NA': 0,
#         np.nan: 0, None: 0
#     },

#     # 3. LandSlope: Lvl Bnk HLS Low → 0 1 2 -1  (you wrote Low → -1 ?)
#     'landslope': {
#         'Lvl': 0, 'Bnk': 1, 'HLS': 2, 'Low': -1,
#         np.nan: 0, None: 0
#     },

#     # 4. Utilities: AllPub NoSewr NoSeWa ELO → 4 3 2 1
#     'utilities': {
#         'AllPub': 4, 'NoSewr': 3, 'NoSeWa': 2, 'ELO': 1,
#         np.nan: 0, None: 0
#     },

#     # 5. LandContour? No — you most likely mean LandSlope: Gtl Mod Sev → 3 2 1
#     'land_slope': {     # usually called LandSlope
#         'Gtl': 3, 'Mod': 2, 'Sev': 1,
#         np.nan: 0, None: 0
#     },

#     # 9. Functional: Typ Min1 Min2 Mod Maj1 Maj2 Sev Sal → 8..1
#     'functional': {
#         'Typ': 8, 'Min1': 7, 'Min2': 6, 'Mod': 5,
#         'Maj1': 4, 'Maj2': 3, 'Sev': 2, 'Sal': 1,
#         np.nan: 8, None: 8   # most common is Typ
#     },

#     # 10. Fence: GdPrv MnPrv GdWo MnWw NA → 4 3 2 1 0
#     'fence': {
#         'GdPrv': 4, 'MnPrv': 3, 'GdWo': 2, 'MnWw': 1, 'NA': 0,
#         np.nan: 0, None: 0
#     },
# }

# # ────────────────────────────────────────────────────────────────
# # Special cases: binary and ternary
# # ────────────────────────────────────────────────────────────────

# def map_binary(col):
#     return col.map({'Y': 1, 'N': 0, np.nan: 0, None: 0, 'NA': 0})

# def map_ternary(col):
#     # Y N P → 1 0 0.5   (P = Partial?)
#     return col.map({'Y': 1, 'N': 0, 'P': 0.5, np.nan: 0, None: 0, 'NA': 0})

# # ────────────────────────────────────────────────────────────────
# # 3-vector encoding for BsmtFinType1 (your custom branched ordinal)
# # ────────────────────────────────────────────────────────────────

# def bsmtfintype1_to_vector(X):
#     # Returns 3-column array: [living, lowq, rec]
#     mapping = {
#         'GLQ': [3, 0, 0],
#         'ALQ': [2, 0, 0],
#         'BLQ': [1, 0, 0],
#         'LwQ': [0, 2, 0],
#         'Unf': [0, 1, 0],
#         'Rec': [0, 0, 2],
#         'NA':  [0, 0, 0],
#         np.nan: [0, 0, 0],
#         None:  [0, 0, 0]
#     }
    
#     def row_to_vec(x):
#         return mapping.get(x, [0, 0, 0])
    
#     vec = X.iloc[:, 0].map(row_to_vec).tolist()   # assuming single column input
#     return np.array(vec)


# # ────────────────────────────────────────────────────────────────
# # Main preprocessing logic
# # ────────────────────────────────────────────────────────────────

# def create_categorical_pipeline(X_train):
#     # Step 1: Identify columns that match each pattern
#     quality_cols_5_1   = [c for c in X_train.columns if X_train[c].dropna()\
#                           .isin(['Ex','Gd','TA','Fa','Po']).any()]
#     bsmt_exposure_cols = [c for c in X_train.columns if X_train[c].dropna()\
#                           .isin(['Gd','Av','Mn','No','NA']).any()]
#     # landslope_cols     = [c for c in X_train.columns if X_train[c].dropna()\
#     #                       .isin(['Lvl','Bnk','HLS','Low']).any()]
#     utilities_cols     = [c for c in X_train.columns if X_train[c].dropna()\
#                           .isin(['AllPub','NoSewr','NoSeWa','ELO']).any()]
#     slope_cols         = [c for c in X_train.columns if X_train[c].dropna()\
#                           .isin(['Gtl','Mod','Sev']).any()]
#     functional_cols    = [c for c in X_train.columns if X_train[c].dropna()\
#                           .isin(['Typ','Min1','Min2','Mod','Maj1','Maj2','Sev','Sal']).any()]
#     fence_cols         = [c for c in X_train.columns if X_train[c].dropna()\
#                           .isin(['GdPrv','MnPrv','GdWo','MnWw','NA']).any()]
#     bsmtfin_cols       = [c for c in X_train.columns if X_train[c].dropna()\
#                           .isin(['GLQ','ALQ','BLQ','Rec','LwQ','Unf','NA']).any()]
    
#     binary_cols        = [c for c in X_train.select_dtypes('object').columns 
#                          if X_train[c].dropna().isin(['Y','N']).all()]
#     ternary_cols       = [c for c in X_train.select_dtypes('object').columns 
#                          if X_train[c].dropna().isin(['Y','N','P']).all()]

# def bsmtfintype1_to_vector(X):
#     # Returns 3-column array: [living, lowq, rec]
#     mapping = {
#         'GLQ': [3, 0, 0],
#         'ALQ': [2, 0, 0],
#         'BLQ': [1, 0, 0],
#         'LwQ': [0, 2, 0],
#         'Unf': [0, 1, 0],
#         'Rec': [0, 0, 2],
#         'NA':  [0, 0, 0],
#         np.nan: [0, 0, 0],
#         None:  [0, 0, 0]
#     }
    
#     def row_to_vec(x):
#         return mapping.get(x, [0, 0, 0])
    
#     vec = X.iloc[:, 0].map(row_to_vec).tolist()   # assuming single column input
#     return np.array(vec)

    
#     # ────────────────────────────────────────────────
#     # Transformers for each group (comment any block to disable)
#     # ────────────────────────────────────────────────

#     transformers = []

#     # 1,2,3,4,5,9,10 — Ordinal mappings
#     for name, mapping in ordinal_mappings.items():
#         cols = locals().get(name + '_cols', [])   # e.g. quality_cols_5_1
#         if cols:
#             transformers.append(
#                 (name, 
#                  Pipeline([
#                      ('impute', SimpleImputer(strategy='most_frequent')),
#                      ('map',    FunctionTransformer(lambda x: x.map(mapping).fillna(0))),
#                  ]), 
#                  cols)
#             )

#     # BsmtFinType1 → 3-vector
#     if bsmtfin_cols:
#         transformers.append(
#             ('bsmtfin_vector', 
#              Pipeline([
#                  ('impute', SimpleImputer(strategy='constant', fill_value='NA')),
#                  ('vector', FunctionTransformer(bsmtfintype1_to_vector)),
#              ]), 
#              bsmtfin_cols)
#         )

#     # Y/N columns → 1/0
#     if binary_cols:
#         transformers.append(
#             ('binary', FunctionTransformer(map_binary), binary_cols)
#         )

#     # Y/N/P columns → 1/0/0.5
#     if ternary_cols:
#         transformers.append(
#             ('ternary', FunctionTransformer(map_ternary), ternary_cols)
#         )

#     # ────────────────────────────────────────────────
#     # Remaining columns: low-cardinality → one-hot, high → drop
#     # ────────────────────────────────────────────────

#     all_processed = set()
#     for _, _, cols in transformers:
#         all_processed.update(cols if isinstance(cols, list) else [cols])

#     remaining_cols = [c for c in X_train.select_dtypes(include='object').columns 
#                       if c not in all_processed]

#     low_card_cols = []
#     high_card_cols = []

#     for col in remaining_cols:
#         n_unique = X_train[col].dropna().nunique()
#         if n_unique <= 15:
#             low_card_cols.append(col)
#         else:
#             high_card_cols.append(col)
#             print(f"Dropping high-cardinality column: {col} ({n_unique} unique values)")

#     if low_card_cols:
#         transformers.append(
#             ('onehot_low', 
#              Pipeline([
#                  ('impute', SimpleImputer(strategy='most_frequent')),
#                  ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
#              ]), 
#              low_card_cols)
#         )

#     # Final preprocessor
#     preprocessor = ColumnTransformer(
#         transformers=transformers,
#         remainder='passthrough'   # numeric columns unchanged
#     )

#     return preprocessor
# #========================================================================================================

# Define model
model = RandomForestRegressor(n_estimators=100, random_state=0) # Your code here

# Check your answer
step_1.a.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [95]:
# Lines below will give you a hint or solution code
#step_1.a.hint()
#step_1.a.solution()

### Part B

Run the code cell below without changes.

To pass this step, you need to have defined a pipeline in **Part A** that achieves lower MAE than the code above.  You're encouraged to take your time here and try out many different approaches, to see how low you can get the MAE!  (_If your code does not pass, please amend the preprocessing steps and model in Part A._)

In [17]:
# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

# Check your answer
step_1.b.check()

MAE: 17614.81993150685


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# Line below will give you a hint
#step_1.b.hint()

# Step 2: Generate test predictions

Now, you'll use your trained model to generate predictions with the test data.

In [18]:
# Preprocessing of test data, fit model
preds_test = my_pipeline.predict(X_test) # Your code here

# Check your answer
step_2.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [19]:
# Lines below will give you a hint or solution code
#step_2.hint()
#step_2.solution()

Run the next code cell without changes to save your results to a CSV file that can be submitted directly to the competition.

In [20]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)

# Submit your results

Once you have successfully completed Step 2, you're ready to submit your results to the leaderboard!  If you choose to do so, make sure that you have already joined the competition by clicking on the **Join Competition** button at [this link](https://www.kaggle.com/c/home-data-for-ml-course).  
1. Begin by clicking on the **Save Version** button in the top right corner of the window.  This will generate a pop-up window.  
2. Ensure that the **Save and Run All** option is selected, and then click on the **Save** button.
3. This generates a window in the bottom left corner of the notebook.  After it has finished running, click on the number to the right of the **Save Version** button.  This pulls up a list of versions on the right of the screen.  Click on the ellipsis **(...)** to the right of the most recent version, and select **Open in Viewer**.  This brings you into view mode of the same page. You will need to scroll down to get back to these instructions.
4. Click on the **Data** tab near the top of the screen.  Then, click on the file you would like to submit, and click on the **Submit** button to submit your results to the leaderboard.

You have now successfully submitted to the competition!

If you want to keep working to improve your performance, select the **Edit** button in the top right of the screen. Then you can change your code and repeat the process. There's a lot of room to improve, and you will climb up the leaderboard as you work.


# Keep going

Move on to learn about [**cross-validation**](https://www.kaggle.com/alexisbcook/cross-validation), a technique you can use to obtain more accurate estimates of model performance!

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/intermediate-machine-learning/discussion) to chat with other learners.*